In [ ]:
%load_ext autoreload
%autoreload 2

# word2vec 

This notebook is equivalent to `demo-word.sh`, `demo-analogy.sh`, `demo-phrases.sh` and `demo-classes.sh` from Google.

## Training

In [ ]:
import word2vec

Run `word2phrase` to group up similar words "Los Angeles" to "Los_Angeles"

In [ ]:
word2vec.word2phrase('text8', 'text8-phrases', verbose=True)

This will create a `text8-phrases` that we can use as a better input for `word2vec`.
Note that you could easily skip this previous step and use the origial data as input for `word2vec`.

Train the model using the `word2phrase` output.

In [ ]:
word2vec.word2vec('text8-phrases', 'text8.bin', size=100, verbose=True)

That generated a `text8.bin` file containing the word vectors in a binary format.

Do the clustering of the vectors based on the trained model.

In [ ]:
word2vec.word2clusters('text8.bin', 'text8-clusters.txt', 100, verbose=True)

That created a `text8-clusters.txt` with the cluster for every word in the vocabulary

## Predictions

In [ ]:
import word2vec

Import the `word2vec` binary file created above

In [ ]:
model = word2vec.load('text8.bin')

We can take a look at the vocabulaty as a numpy array

In [ ]:
model.vocab


Or take a look at the whole matrix

In [ ]:
model.vectors.shape

In [ ]:
model.vectors

We can retreive the vector of individual words

In [ ]:
model['dog'].shape

In [ ]:
model['dog']

We can do simple queries to retreive words similar to "socks" based on cosine similarity:

In [ ]:
indexes, metrics = model.cosine('computer_science')
indexes, metrics

This returned a tuple with 2 items:
1. numpy array with the indexes of the similar words in the vocabulary
2. numpy array with cosine similarity to each word

Its possible to get the words of those indexes

In [ ]:
model.vocab[indexes]

There is a helper function to create a combined response: a numpy [record array](http://docs.scipy.org/doc/numpy/user/basics.rec.html)

In [ ]:
model.generate_response(indexes, metrics)

Is easy to make that numpy array a pure python response:

In [ ]:
model.generate_response(indexes, metrics).tolist()

### Phrases

Since we trained the model with the output of `word2phrase` we can ask for similarity of "phrases"

In [ ]:
indexes, metrics = model.cosine('portland_oregon')
model.generate_response(indexes, metrics).tolist()

### Analogies

Its possible to do more complex queries like analogies such as: `king - man + woman = queen` 
This method returns the same as `cosine` the indexes of the words in the vocab and the metric

In [ ]:
indexes, metrics = model.analogy(pos=['computer', 'electrical'], neg=['science'], n=10)
indexes, metrics

In [ ]:
model.generate_response(indexes, metrics).tolist()

### Clusters

In [ ]:
clusters = word2vec.load_clusters('text8-clusters.txt')

We can see get the cluster number for individual words

In [ ]:
clusters['dog']

We can see get all the words grouped on an specific cluster

In [ ]:
clusters.get_words_on_cluster(90).shape

In [ ]:
clusters.get_words_on_cluster(90)[:10]

We can add the clusters to the word2vec model and generate a response that includes the clusters

In [ ]:
model.clusters = clusters

In [ ]:
indexes, metrics = model.analogy(pos=['paris', 'germany'], neg=['france'], n=10)

In [ ]:
model.generate_response(indexes, metrics).tolist()